In [1]:
library('SGL')
library('grpregOverlap')
library('MASS')
library('glmnet')
library('parallel')

Loading required package: Matrix
Loading required package: grpreg

Attaching package: ‘MASS’

The following object is masked from ‘package:grpregOverlap’:

    select

The following object is masked from ‘package:grpreg’:

    select

Loading required package: foreach
Loaded glmnet 2.0-16



In [2]:
numPheno <- 1
numGenes <- 2999
numSamples <- 500
numPaths <- 20

In [3]:
readDF <- function(i, dataFileBase='/homes/gws/psturm/simulatedData/corLatentData/df%i.csv') {
    ret <- list()
    
    df <- read.csv(sprintf(dataFileBase, i), header=TRUE)
    df = as.data.frame(df)
    df_t = as.data.frame(t(df))
    colnames(df_t) = c(paste('p', sep='', 1:numPheno), paste('g', sep='', 1:numGenes))
    data_mat = df_t[1:numSamples, ]

    phenotype_genes <- which(df[['phenotype_genes']][-1] != 0)
    y <- data_mat$p1
    x <- data.matrix(data_mat[, paste('g', sep='', 1:numGenes)])

    bin_path_mat <- df[paste('pathway', 0:(numPaths-1), sep='')]
    bin_path_mat <- bin_path_mat[-1, ]

    group_index <- integer(numGenes) - 1
    group_obj <- which(bin_path_mat == 1, arr.ind=T)
    group_index[as.vector(group_obj[, 1])] <- as.vector(group_obj[, 2])
    
    ret$x <- x
    ret$y <- y
    ret$group_index <- group_index
    
    ret$phenotype_genes <- phenotype_genes
    
    ret$group_df <- lapply(unique(group_index), function(o, gi) { 
                        paste('g', which(gi == o), sep='')
                        }, 
                group_index)
    return(ret)
}

In [4]:
getCounts <- function(coeffs, reference_index) {
    discovered_genes <- which(coeffs != 0)
    discovered_genes <- discovered_genes[order(abs(coeffs[discovered_genes]), decreasing=TRUE)]
    num_discovered <- length(discovered_genes)
    total_count <- cumsum(discovered_genes %in% reference_index)
    total_count <- total_count / length(reference_index)
    return(total_count)
}

In [ ]:
numReps = 49 #should be 49
cumulative_overlap_counts <- mclapply(0:numReps, function(i) {
    cat(sprintf("Started rep:\t%i, at time %s\n", i, Sys.time()), file="trainReg_cor_progress.txt", append=TRUE)
    
    overlap_counts <- list()
    
    ret <- readDF(i)
    x <- ret$x
    y <- ret$y
    group_index <- ret$group_index
    phenotype_genes <- ret$phenotype_genes
    group_df <- ret$group_df
    
#   ELASTIC NET
    elasticnet <- glmnet(x, y, alpha = 0.200000, lambda = 0.106429)
    coef_enet <- as.matrix(coef(elasticnet, s=elasticnet$lambda))[-1]
    count_enet <- getCounts(coef_enet, phenotype_genes)
    overlap_counts$elastic_net <- count_enet

#   SPARSE GROUP LASSO
    sparse_gl <- SGL(data=list(x=x, y=y), index=group_index, lambda = 0.0175799340984784, type="linear", alpha=0.5)
    coef_sgl  <- sparse_gl$beta
    count_sgl <- getCounts(coef_sgl, phenotype_genes)
    overlap_counts$sparse_gl <- count_sgl
    
#   OVERLAPPING GROUP LASSO
#     overlap_gl <- grpregOverlap(x, y, group_df, penalty="grLasso", alpha=1, lambda=0.1)
#     coef_overlap <- as.matrix(overlap_gl$beta)[-1]
#     count_overlap <- getCounts(coef_overlap, phenotype_genes)
#     overlap_counts$overlap_gl <- count_overlap
    cat(sprintf("Ended rep:\t%i, at time %s\n", i, Sys.time()), file="trainReg_cor_progress.txt", append=TRUE)
    
    overlap_counts
}, mc.cores=10)

In [ ]:
df_from_counts <- function(counts, name) {
    name_list <- lapply(counts, `[[`, name)
    maxlength <- max(lengths(name_list))
    name_list <- lapply(name_list, function(o, m) {
    o <- if (length(o) < m) c(o, integer(m - length(o)) + o[length(o)]) else o
}, maxlength)
    name_list <- do.call('cbind', name_list)
    name_list <- as.data.frame(name_list)
    colnames(name_list) <- paste('run', 1:length(name_list), sep='')
    name_list
}

In [ ]:
elastic_net_df <- df_from_counts(cumulative_overlap_counts, 'elastic_net')
sparse_gl_df   <- df_from_counts(cumulative_overlap_counts, 'sparse_gl')
# overlap_gl_df  <- df_from_counts(cumulative_overlap_counts, 'overlap_gl')

In [ ]:
baseSaveDir <- '../../DataFrames/corLatentRegression/%s'
write.csv(elastic_net_df, file=sprintf(baseSaveDir, 'elastic_net.csv'), row.names=FALSE)
write.csv(sparse_gl_df, file=sprintf(baseSaveDir, 'sparse_gl.csv'), row.names=FALSE)
# write.csv(overlap_gl_df, file=sprintf(baseSaveDir, 'overlap_gl.csv'), row.names=FALSE)

In [ ]:
save(cumulative_overlap_counts, file='/projects/leelab3/psturm/cor_sim_reg_counts.RData')